#Concat 

In [ ]:
import pandas as pd
import os

Our `read_csv` function.

In [ ]:
# %load utils.py

import pandas as pd

def read_csv(filename):
    df = pd.read_csv(filename, skiprows=8)
    df.rename(columns={'# timestamp': 'timestamp'}, inplace=True)
    df['timestamp'] = pd.to_datetime(df['timestamp'])  #might not want this here
    df.set_index('timestamp', inplace=True)
    return df

In [ ]:
filenames = ['data/measured_real_power.csv',
             'data/measured_reactive_power.csv']

Concat

In [ ]:
df1 = read_csv(filenames[0])
df2 = read_csv(filenames[1])

df = pd.concat([df1, df2], axis=1) # stack side by side

In [ ]:
df1.shape, df2.shape, df.shape

In [ ]:
len(set(df.columns))

In [ ]:
all(df1.columns == df2.columns)

So now we need to capture some additional information.

In [ ]:
def read_csv(filename):
    df = pd.read_csv(filename, skiprows=8)
    df.rename(columns={'# timestamp': 'timestamp'}, inplace=True)
    df.timestamp = pd.to_datetime(df.timestamp)
    df.set_index('timestamp', inplace=True)
    prefix = os.path.basename(filename).replace('.csv','')
    df.columns = [ "{}:{}".format(prefix, x) for x in df.columns ]
    return df

df = pd.concat([ read_csv(x) for x in filenames ], axis=1)
len(set(df.columns))

## Glob 

In [ ]:
import glob

In [ ]:
filenames = glob.glob('data/neighborhood/outputFiles/*.csv')

def read_dir(filenames):
    return pd.concat([ read_csv(x) for x in filenames ], axis=1)

read_dir(filenames).shape

Takes a while... Can we do better?

In [ ]:
import cProfile

cProfile.run('read_dir(filenames)')

In [ ]:
%%writefile utils.py
import pandas as pd
import os

def read_csv(filename):
    df = pd.read_csv(filename, skiprows=8)
    df.rename(columns={'# timestamp': 'timestamp'}, inplace=True)
    #df.timestamp = pd.to_datetime(df.timestamp) #hold off for now...
    df.set_index('timestamp', inplace=True)
    prefix = os.path.basename(filename).replace('.csv','')
    df.columns = [ "{}:{}".format(prefix, x) for x in df.columns ] 
    return df

def read_dir(filenames):
    df = pd.concat([ read_csv(x) for x in filenames ], axis=1)
    df.index = pd.to_datetime(df.index)
    return df

In [ ]:
read_dir(filenames).shape